In [12]:
import os
import sys
import uuid

pipeline_dir = os.path.join(os.getcwd(), '..')
if pipeline_dir not in sys.path:
    sys.path.append(pipeline_dir)

In [2]:
from pipeline import Pipeline, events, resources

# Getting Started
You can create a new project with the `pipeline-create <new_directory>` which generates a template project in a new directory.  Edit the `handler.py` file to build your pipeline.

# Building a Pipeline
Pipelines are made up of lambda functions, events, resources, and services.  Let us start by defining an empty pipeline by inheriting from the base class.

In [3]:
class MyFirstPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(name="my-first-pipeline")

## Define a Function
We can define a lambda function by defining a class method decorated with an event trigger.  The decorator determines how the lambda function is executed.  Let's write a simple "Hello World" function triggered with a GET request.

In [4]:
class MyFirstPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(name="my-first-pipeline")
    
    @events.http(method="get", path="helloworld", cors="true")
    def hello_world(self, event, context):
        print("Hello World!")

## Define a Resource
All that is required to build a working pipeline is a `pipeline.Pipeline` object with at least one lambda function decorated by an event trigger.  If we want our pipeline to interact with other AWS resources we may define a resource.  Let's define a DynamoDB table which we will use to store messages:

In [5]:
class MyTable(resources.DynamoDB):
    
    def __init__(self):
        super().__init__()
        self.add_attribute('id', 'S')
        self.add_key('id', 'HASH')

We will also edit our original lambda function to take user input in the GET request.  We can register our newly created resource by instantiating and passing into the call to `super`.  Additionally, resources provide helper methods for interacting with the resource.  The DynamoDB resource, for example, inherits the `put`, `get`, `delete`, and `list` methods.

In [8]:
table = MyTable()

class MyFirstPipeline(Pipeline):
    
    def __init__(self):
        super().__init__(name="my-first-pipeline",
                         resources=[table])
    
    @events.http(method="get", path="helloworld/{message}", cors="true")
    def hello_world(self, event, context):
        item = {'id': str(uuid.uuid1),
                'message': event['message']
               }
        # Use helper method
        table.put(item)

## Packaging
Now that your pipeline is built, we need to do some final packaging before we can deploy.  Because lambda can't call functions inside of classes, we need to expose each decorated method.  Additionally, we need to add a deploy wrapper for deploying the pipeline.  Once finished your project should look something like `cognition-pipeline/examples/quickstart/`.

In [9]:
# Instantiate your pipeline
pipeline = MyFirstPipeline()

# Lambda func entrypoint
hello_world = pipeline.hello_world

# Deploy wrapper
def deploy():
    pipeline.deploy()

## Requirements
Requirements may either be added to `requirements.txt` in the pipeline directory or handled through the use of services.

## Deploying
Once your pipeline is packaged it is time to deploy!  Use the `pipeline-deploy <pipeline_directory>` to generate a `serverless.yml` file which may be deployed with Serverless Framework.

## Testing
Cognition-pipeline integrates with unittests by providing an object-oriented framework for interacting with your functions and resources.  Simply pass your instantiated pipeline into the standard `unittest.TestCase.setUp` method and test your functions!

In [10]:
import unittest
import time

class MyFirstPipelineTestCases(unittest.TestCase):

    def setUp(self):
        self.pipeline = pipeline

    def test_hello_world(self):
        func = self.pipeline.functions['hello_world']
        table = self.pipeline.resources['QuickstartTable']
        # Invoke your lambda function (creates a new entry in DynamoDB Table)
        func.invoke('test_entry')
        # Give the table some time to update
        time.sleep(5)
        # Confirm the new entry using a DynamoDB table scan
        response = table.list()
        self.assertEqual(len(response), 1)
        self.assertEqual(response[0]['message'], 'test_entry')
        # Delete the new entry with the entry's id
        table.delete(response[0]['id'])